# Cleaning and transforming data

Files needed = ('atussum_2017.dat', 'movies.csv')

In this notebook we will continue practicing techniques for manipulating data into forms that are amenable to analysis. We will cover: 

1. `.replace()` and `unique()` for recoding variables
2. `.map()` and `.applymap()` for working element-wise on DataFrames
3. String methods for working with strings in DataFrames

We will also learn about the American Time Use Survey. 

In [33]:
import pandas as pd                 # pandas for data handling
import matplotlib.pyplot as plt     # matplotlib for plots
import numpy as np                  # numpy for numerical methods 

## American Time Use Survey (ATUS)
The Bureau of Labor Statistics oversees the [American Time Use Survey](https://www.bls.gov/tus/overview.htm), which asks a sample of Americans to complete detailed diaries keeping track of each minute of their day. 

Follow this link [www.bls.gov/tus/data/datafiles_2017.htm](https://www.bls.gov/tus/data/datafiles_2017.htm) to the page for the 2017 survey. Download the **ATUS 2017 Activity summary file (zip)** file located in the **2017 Basic ATUS Data Files** section of the page. Alternatively, download it directly [www.bls.gov/tus/datafiles/atussum_2017.zip](https://www.bls.gov/tus/datafiles/atussum_2017.zip). 

Unzip the file. We are looking for `atussum_2017.dat`. It is a comma-separated file (even though it has a '.dat' extension). Let's get it loaded.

### Variables

This data set has 421 variables! That's too many for us today. Let's just keep a few. 

The demographic variables are all uppercase letters. The time variables are of the form 'txxyyzz' where xx is the major category code, yy is the second-tier code, and zz is the third-tier code. ([docs](https://www.bls.gov/tus/lexicons/lexiconwex2017.pdf))

Let's keep some demographic data and some data about working and sleeping. 

In [27]:
variables = {'TEAGE':'age', 'TESEX':'sex', 'PTDTRACE':'race', 'PEEDUCA':'edu', 'GTMETSTA':'metro', 'TELFS':'employ', 
 'TUDIARYDAY':'day', 't050101':'work_main', 't050102':'work_other', 't010101':'sleep', 't050201':'work_soc', 't010102':'no_sleep'}

In [28]:
import pandas as pd
atus_small = pd.read_csv('atussum_2017.dat', usecols=variables.keys())


In [29]:
atus_small.rename(columns=variables, inplace=True)

In [5]:
atus_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10223 entries, 0 to 10222
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   age         10223 non-null  int64
 1   sex         10223 non-null  int64
 2   edu         10223 non-null  int64
 3   race        10223 non-null  int64
 4   metro       10223 non-null  int64
 5   employ      10223 non-null  int64
 6   day         10223 non-null  int64
 7   sleep       10223 non-null  int64
 8   no_sleep    10223 non-null  int64
 9   work_main   10223 non-null  int64
 10  work_other  10223 non-null  int64
 11  work_soc    10223 non-null  int64
dtypes: int64(12)
memory usage: 958.5 KB


Okay, let's start transforming the data to get it ready for analysis. 

### A word on loops
We have seen many places where a loop saved us time and effort. It might seem natural to use a loop to perform an operation on each element of a column or a DataFrame. 

In general, however, we want to avoid this. Instead, we have used pandas vectorized operations such as 
```python
x['new_var'] = x['var_1'] / x['var_2']
```
to perform element-wise division. Using pandas' native operations is much (much much) faster than looping over the rows of a DataFrame. A lot of optimization has been written into the native functions that is not there when we loop ourselves. If you find yourself looping over the rows of a DataFrame, take a step back and think about other ways forward. 

Fortunately, pandas provides methods that let us do very complex and very general operations to a DataFrame without resorting to a loop. We consider a few of these below. 

### replace( ) 

The sex variable is coded 1 for male and 2 for female. I do not want to have to remember that!

The `replace( )` method replaces one value for another. One syntax is 
```
atus_small['sex'] = atus_small['sex'].replace(1, 'male')
```
but a more powerful one passes a dict or a list.
```
atus_small['sex'] = atus_small['sex'].replace({1:'male', 2:'female'})
```

In [30]:
sex_codes = {1:'male', 2:'female'}
atus_small['sex'] = atus_small['sex'].replace(sex_codes)
atus_small.head()

,age,sex,edu,race,metro,employ,day,sleep,no_sleep,work_main,work_other,work_soc
0,34,female,39,2,1,1,1,728,0,450,0,0
1,28,female,40,1,1,5,7,385,0,0,0,0
2,15,male,35,3,1,5,4,570,0,0,0,0
3,46,male,39,1,1,1,2,525,0,480,0,0
4,85,male,44,1,1,1,7,756,0,0,0,0


### unique( )
Let's code race, too. What codes are in our data? The method `unique( )` returns the unique values.

In [7]:
races = atus_small['race'].unique()
races.sort()
print(races)

[ 1  2  3  4  5  6  7  8  9 10 11 13 16 18]


In [8]:
# I looked up the codes in the documentation...

race_codes = {1:'white', 2:'black', 3:'native_am', 4:'asian', 5:'hawaiian', 6:'wh_bl', 
              7:'wh_na', 8:'wh_as', 9:'wh_ha', 10:'bl_na', 11:'bl_as', 13:'na_as', 16:'wh_bl_as', 18:'wh_as_ha'}

In [9]:
atus_small['race'] = atus_small['race'].replace(race_codes)

In [10]:
atus_small.head()

,age,sex,edu,race,metro,employ,day,sleep,no_sleep,work_main,work_other,work_soc
0,34,female,39,black,1,1,1,728,0,450,0,0
1,28,female,40,white,1,5,7,385,0,0,0,0
2,15,male,35,native_am,1,5,4,570,0,0,0,0
3,46,male,39,white,1,1,2,525,0,480,0,0
4,85,male,44,white,1,1,7,756,0,0,0,0


### Apply a function to a Series \[a single column\]: map( )

We can apply functions to the individual elements in a column, too. The `map()` functions handles this for us. It applies the given function to each element of the column. These can be built-in functions, or user-defined functions. This is quite powerful. We can write a function that performs a complicated transformation and apply to each element of a column in one simple line.  

Let's define a function that converts minutes to hours. 

In [31]:
def minutes_to_hours(x):
    return x/60

In [32]:
atus_small.describe()

,age,edu,race,metro,employ,day,sleep,no_sleep,work_main,work_other,work_soc
count,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000
mean,49.876846,40.685024,1.371515,1.169422,2.564805,3.931038,526.711728,5.434413,148.908442,4.536535,0.126675
std,18.008395,2.807664,1.013168,0.403051,1.907025,2.352525,138.143394,30.543610,235.790188,39.093847,9.210473
min,15.000000,31.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.000000,39.000000,1.000000,1.000000,1.000000,1.000000,450.000000,0.000000,0.000000,0.000000,0.000000
50%,50.000000,40.000000,1.000000,1.000000,1.000000,4.000000,515.000000,0.000000,0.000000,0.000000,0.000000
75%,64.000000,43.000000,1.000000,1.000000,5.000000,6.000000,600.000000,0.000000,353.000000,0.000000,0.000000
max,85.000000,46.000000,18.000000,3.000000,5.000000,7.000000,1413.000000,980.000000,1288.000000,914.000000,830.000000


Now apply `map( )` to the work column. 

\[This example is a bit contrived. We could have just as easily done `atus_small['work_main']/60`. But that wouldn't have given us practice with `map( )`.\] 

In [16]:
atus_small['work_main'] = atus_small['work_main'].map(minutes_to_hours)
atus_small.head()

,age,sex,edu,race,metro,employ,day,sleep,no_sleep,work_main,work_other,work_soc
0,34,female,39,black,1,1,1,728,0,0.125000,0,0
1,28,female,40,white,1,5,7,385,0,0.000000,0,0
2,15,male,35,native_am,1,5,4,570,0,0.000000,0,0
3,46,male,39,white,1,1,2,525,0,0.133333,0,0
4,85,male,44,white,1,1,7,756,0,0.000000,0,0


### Apply a function to a DataFrame: applymap( )
If we want to apply the same function to each element in several columns of a DataFrame (rather than a Series) we use `applymap( )`. It works the same way, applying the function to each element. 

\[I am not sure why the developers of pandas created separate methods for a Series vs a DataFrame. My best guess is that each method takes advantage of Series- or Dataframe-specific functionality for better performance.\]

In [17]:
# We can map to several columns at once.
atus_small[['work_other', 'sleep', 'no_sleep']] = atus_small[['work_other', 'sleep', 'no_sleep']].applymap(minutes_to_hours)
atus_small.head()

C:\Users\local_rhong22\Temp\2\ipykernel_980\133808933.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  atus_small[['work_other', 'sleep', 'no_sleep']] = atus_small[['work_other', 'sleep', 'no_sleep']].applymap(minutes_to_hours)


,age,sex,edu,race,metro,employ,day,sleep,no_sleep,work_main,work_other,work_soc
0,34,female,39,black,1,1,1,12.133333,0.0,0.125000,0.0,0
1,28,female,40,white,1,5,7,6.416667,0.0,0.000000,0.0,0
2,15,male,35,native_am,1,5,4,9.500000,0.0,0.000000,0.0,0
3,46,male,39,white,1,1,2,8.750000,0.0,0.133333,0.0,0
4,85,male,44,white,1,1,7,12.600000,0.0,0.000000,0.0,0


`.map()` and `.applymap()` are very powerful methods. They can do a lot more than we have covered and they can be used to do very complex manipulations. We won't need those advanced capabilities, but they are out there if you someday do.   

In [18]:
atus_small.describe()

,age,edu,metro,employ,day,sleep,no_sleep,work_main,work_other,work_soc
count,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000,10223.000000
mean,49.876846,40.685024,1.169422,2.564805,3.931038,8.778529,0.090574,0.041363,0.075609,0.126675
std,18.008395,2.807664,0.403051,1.907025,2.352525,2.302390,0.509060,0.065497,0.651564,9.210473
min,15.000000,31.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.000000,39.000000,1.000000,1.000000,1.000000,7.500000,0.000000,0.000000,0.000000,0.000000
50%,50.000000,40.000000,1.000000,1.000000,4.000000,8.583333,0.000000,0.000000,0.000000,0.000000
75%,64.000000,43.000000,1.000000,5.000000,6.000000,10.000000,0.000000,0.098056,0.000000,0.000000
max,85.000000,46.000000,3.000000,5.000000,7.000000,23.550000,16.333333,0.357778,15.233333,830.000000


## Top Hat Practice Exercise: Transform
Take a few minutes and try the following. Feel free to chat with those around you if you get stuck. 

1. `edu` in 'atus_small' holds the highest level of education obtained. It can take values between 31 and 46. Covert the numeric values to labels. 

   * 39 = 'high school'
   * 40 = 'some college'
   * 41 & 42 are 'associate'
   * 43 = 'bachelor'
   * 44 = 'master'
   * 45 = 'prof'
   * 46 = 'phd'

In [19]:
atus_small.edu

0        39
1        40
2        35
3        39
4        44
         ..
10218    42
10219    39
10220    40
10221    43
10222    46
Name: edu, Length: 10223, dtype: int64

In [30]:
codes = {39 :'high school', 40 : 'some college', 41 : 'associate', 42 : 'associate',43 : 'bachelor', 44 : 'master', 45 : 'prof', 
             46 :'phd'}
atus_small['edu'] = atus_small['edu'].replace(codes)
atus_small.head(2)

,age,sex,edu,race,metro,employ,day,sleep,no_sleep,work_main,work_other,work_soc
0,34,female,high school,black,1,1,1,12.133333,0.0,0.125,0.0,0
1,28,female,some college,white,1,5,7,6.416667,0.0,0.000,0.0,0


2. Print out a list of the unique values in the column 'edu'. 

In [26]:
atus_small.edu.unique()

array(['high school', 'some college', 35, 'master', 'bachelor', 36,
       'prof', 'associate', 'phd', 37, 34, 33, 38, 31, 32], dtype=object)

3. Apply the `minutes_to_hours` function to the `work_soc` variable. This variable is the time spent 'Socializing, relaxing, and leisure as part of job'. I have no idea what kind of jobs makes you relax and take leisure. 

In [28]:
atus_small

,age,sex,edu,race,metro,employ,day,sleep,no_sleep,work_main,work_other,work_soc
0,34,female,high school,black,1,1,1,12.133333,0.0,0.125000,0.0,0
1,28,female,some college,white,1,5,7,6.416667,0.0,0.000000,0.0,0
2,15,male,35,native_am,1,5,4,9.500000,0.0,0.000000,0.0,0
3,46,male,high school,white,1,1,2,8.750000,0.0,0.133333,0.0,0
4,85,male,master,white,1,1,7,12.600000,0.0,0.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10218,36,female,associate,white,1,1,3,7.000000,0.0,0.134722,0.0,0
10219,28,male,high school,white,1,1,4,6.500000,0.0,0.137500,0.0,0
10220,56,male,some college,white,1,1,5,9.250000,0.0,0.134722,0.0,0
10221,36,female,bachelor,white,1,2,7,9.416667,0.0,0.000000,0.0,0


In [31]:
atus_small['work_soc'] = atus_small['work_soc'].map(minutes_to_hours)


4. Create a column named 'work' that is the sum of 'work_main' and 'work_other'.

5. Create a histogram of 'work' for only those observations with work>0. 

### Extra practice
If you finish early, try rewriting your solution to part 1, but code values less than 39 as "less than high school".

## String methods
We have seen some of these before, when we learned about strings. These are analogous to the string methods in standard python, but they have been optimized for DataFrames. These *vectorized string methods*, work element-wise over an entire column. The method call looks like

```python
data['var'].str.method()
```

where `.str.method( )` is the method we are applying. A list of vectorized string methods is on page 218 in McKinney and in the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#method-summary). Below, we try a few out. 

### MovieLens data set

We are going to work with the [MovieLens](https://grouplens.org/datasets/movielens/) *ml-latest-small* dataset. The GroupLens organization released this data. It is meant to help build recommendation algorithms, like the ones you see in Netflix or Spotify. \[In 2006, [Netflix started a contest](https://en.wikipedia.org/wiki/Netflix_Prize), with a $1 mil. reward, for an algorithm that could beat their own.\] They have other ratings datasets, too, on music, jokes, and books.  

An observation is a movie.

In [34]:
movies = pd.read_csv('movies.csv')
movies.sample(10)

,movieId,title,genres
2421,3219,Pacific Heights (1990),Mystery|Thriller
1076,1396,Sneakers (1992),Action|Comedy|Crime|Drama|Sci-Fi
2200,2925,"Conformist, The (Conformista, Il) (1970)",Drama
7218,73160,Sorority Babes in the Slimeball Bowl-O-Rama (1...,Comedy|Horror
7661,88699,Death Race 2 (2010),Action|Sci-Fi|Thriller
9171,148888,Zoolander 2 (2016),Comedy
8431,111617,Blended (2014),Comedy
5982,36519,Transporter 2 (2005),Action|Crime|Thriller
872,1161,"Tin Drum, The (Blechtrommel, Die) (1979)",Drama|War
2736,3674,For the Love of Benji (1977),Adventure|Children|Comedy|Drama


### str.contains( )
The genres are mixed together. Let's get all the comedies. The `.contains( )` method returns a bool Series with True for observations in which the string contains the search term.

In [ ]:
movies['genres'].str.contains('Comedy')

In [4]:
print(movies.shape)
comedies = movies[movies['genres'].str.contains('Comedy')]
print(comedies.shape)

(9742, 3)
(3756, 3)


### str.split( )
This method splits the string up at the delimiter that is passed to `.split( )`. It returns a list of each chunk that falls between the delimiter. 

This could be useful processing data that come in the form: last,first or city,state. 

In [37]:
# The movie genres are separated with the '|' character. 
# Remember, DataFrames can have columns of lists...

movies['genre_split'] = movies['genres'].str.split('|')       # '|' deliminiter  
movies.head()

,movieId,title,genres,genre_split
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),Comedy,[Comedy]


In [38]:
movies.loc[0,'genre_split']

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

### str.join ( )
Put strings together. Separate the pieces with a delimiter of your choosing. 

In [39]:
movies['with_colons'] = movies['genre_split'].str.join('::')
movies.sample(5)

,movieId,title,genres,genre_split,with_colons
6840,61394,"Onion Movie, The (2008)",Comedy,[Comedy],Comedy
2285,3032,"Omega Man, The (1971)",Action|Drama|Sci-Fi|Thriller,"[Action, Drama, Sci-Fi, Thriller]",Action::Drama::Sci-Fi::Thriller
1618,2162,"NeverEnding Story II: The Next Chapter, The (1...",Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]",Adventure::Children::Fantasy
7238,73808,"Chaser, The (Chugyeogja) (2008)",Crime|Drama|Thriller,"[Crime, Drama, Thriller]",Crime::Drama::Thriller
8374,109416,Bring It On: Fight to the Finish (2009),Comedy,[Comedy],Comedy


## Top Hat Practice Exercise: String Transforms

Take a few minutes and try the following. Feel free to chat with those around you if you get stuck. I am here, too. 

Our data does not have a column for the year the movie was released. Let's create one. The year the movie was released is in the title string.    

0. Reload 'movies.csv'
1. Use `.str.strip()` ([docs](https://pandas.pydata.org/pandas-docs/version/0.24.2/reference/api/pandas.Series.str.strip.html)) to remove any leading or trailing spaces from 'title'.

1. Extract the four-digit year from the titles and put them into a new column named 'year'.  

Notice that the year, including the parentheses, is always the last 6 digits of the title. You might try `str.slice()` and work with negative indexes to count from the end of 'title'. 

If there is any extra space at the end of a title, it will mess up my algorithm! That's why we strip the extra spaces first. 

In [16]:
movies = pd.read_csv('movies.csv')
movies['title'] = movies['title'].str.strip()


In [18]:
movies['year'] = movies['title'].str.slice(-5,-1)
movies

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017
9739,193585,Flint (2017),Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018


3. There are 12 movies that do not have a year in their title. Find them in your DataFrame. You might try the `str.isdigit()` [(doc)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.isdigit.html#pandas.Series.str.isdigit) method to see if the year you extracted in step 2. is numeric. 

In [19]:
movies.loc[~movies['year'].str.isdigit()]             # ~ not a digit in the year col

,movieId,title,genres,year
6059,40697,Babylon 5,Sci-Fi,lon
9031,140956,Ready Player One,Action|Sci-Fi|Thriller,r On
9091,143410,Hyena Road,(no genres listed),Roa
9138,147250,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),atso
9179,149334,Nocturnal Animals,Drama|Thriller,imal
9259,156605,Paterson,(no genres listed),erso
9367,162414,Moonlight,Drama,ligh
9448,167570,The OA,(no genres listed),he O
9514,171495,Cosmos,(no genres listed),osmo
9515,171631,Maria Bamford: Old Baby,(no genres listed),Bab


4. For the movies without a year, set the 'year' variable to `np.nan`. Use `.unique()` to check that all the 'year' values are either numbers (as strings) or `nan`.

`np.nan` is the "not a number" object. Pandas will interpret that it as a missing value. 

In [20]:
movies.loc[~movies['year'].str.isdigit(),'year']   = np.nan      #set those to missing
movies['year'].unique()

array(['1995', '1994', '1996', '1976', '1992', '1967', '1993', '1964',
       '1977', '1965', '1982', '1990', '1991', '1989', '1937', '1940',
       '1969', '1981', '1973', '1970', '1955', '1959', '1968', '1988',
       '1997', '1972', '1943', '1952', '1951', '1957', '1961', '1958',
       '1954', '1934', '1944', '1960', '1963', '1942', '1941', '1953',
       '1939', '1950', '1946', '1945', '1938', '1947', '1935', '1936',
       '1956', '1949', '1932', '1975', '1974', '1971', '1979', '1987',
       '1986', '1980', '1978', '1985', '1966', '1962', '1983', '1984',
       '1948', '1933', '1931', '1922', '1998', '1929', '1930', '1927',
       '1928', '1999', '2000', '1926', '1919', '1921', '1925', '1923',
       '2001', '2002', '2003', '1920', '1915', '1924', '2004', '1916',
       '1917', '2005', '2006', '1902', nan, '1903', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '1908'], dtype=object)

5. Convert the year variable to datetime. Check your dtypes to verify. 

In [23]:
movies['year'] =pd.to_datetime(movies['year'])
movies.dtypes

movieId             int64
title              object
genres             object
year       datetime64[ns]
dtype: object

## Bonus practice to try at home

If you finished early, compute the number of movies in the DataFrame for each year. You might try `.groupby()`.

1. Which year has the most observations?

Here is something I have not figured out yet...

2. How many movies are there in each decade? i.e., 1900&ndash;1909, 1910&ndash;1919, 1920&ndash;1929...